# Actividad 1:
# Optimización inteligente de modelos predictivos en salud: Predicción de diabetes con ajuste de hiperparámetros

## Objetivo
Desarrollar un modelo de clasificación para predecir diabetes tipo II utilizando el dataset Pima Indians, aplicando y comparando técnicas de ajuste de hiperparámetros (Grid Search, Random Search y Optimización Bayesiana) para encontrar la mejor configuración del modelo y analizar su impacto en el rendimiento.

**Dataset utilizado:**  
[Pima Indians Diabetes Dataset](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)

**Variables:**
- Pregnancies (Embarazos)
- Glucose (Glucosa)
- BloodPressure (Presión arterial)
- SkinThickness (Espesor piel)
- Insulin (Insulina)
- BMI (Índice masa corporal)
- DiabetesPedigreeFunction (Historial familiar)
- Age (Edad)
- Outcome (Resultado: 1 = diabético, 0 = no diabético)

---

### Estructura del Notebook:
1. Metodología.
2. Importación de librerias a utilizar.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo
1. **Carga y exploración de datos:**
   - Análisis de dimensiones y tipos de variables.
   - Estadísticas descriptivas básicas.

2. **Preprocesamiento:**
   - Detección y tratamiento de valores anómalos. No se trataron mas valores anómalos como valores fuera de un rango médico debido a que se desconoce en este trabajo como son estos rangos de valores en la población específica de donde fueron obtenidos, por lo que los únicos valores anómalos tratados fueron los ceros de variables que, biológicamente, es imposible que sean cero, los cuales fueron transformados a mediana ya que es una forma "estándar" de tratar los datos con valores cero.
   - Escalado de variables numéricas (StandardScaler).
   - División estratificada del dataset (70% entrenamiento - 30% prueba).

3. **Modelado y optimización:**
   - Modelo base: Random Forest sin ajuste
   - Técnicas de optimización:
     - Grid Search (búsqueda exhaustiva en malla paramétrica)
     - Random Search (muestreo aleatorio de parámetros)
     - Optimización Bayesiana (Optuna con 50 trials)
   - Métricas de evaluación: 
      - F1-Score: es la media armónica de la precisión y el recall, especialmente útil cuando hay un desequilibrio de clases en los datos.
      - Precisión: mide la exactitud de las predicciones positivas del modelo. "¿Cuantos predicciones positivas son realmente positivas?"
      - Recall: mide la capacidad del modelo para identificar todas las instancias positivas reales. "¿De todas las instancias positivas, cuantas predijo correctamente?"
      - AUC: métrica que mide la capacidad de un modelo de clasificación para distinguir entre clases. La curva ROC traza la tasa de verdaderos positivos (Recall) contra la tasa de falsos positivos (FPR) en varios umbrales de clasificación.

4. **Evaluación comparativa:**
   - Análisis de rendimiento (métricas).
   - Comparación de tiempos de ejecución.
   - Visualización de resultados.

### Configuración clave
- **Modelo:** Random Forest Classifier
- **Hiperparámetros optimizados:**
  - n_estimators (50-200)
  - max_depth (5-30)
  - min_samples_split (2-20)
  - min_samples_leaf (1-10)
  - max_features (sqrt/log2)
- **Manejo de desbalanceo:** `class_weight='balanced'`
- **Métrica de optimización:** AUC-ROC

### Visualización de resultados

- Comparación de métricas:
    - Gráfico de barras con F1-Score, Precisión, Recall y AUC por método.
- Tiempos de ejecución:
    - Gráfico de barras con escala logarítmica comparando tiempos.
- Curvas ROC:
    - Comparación visual del rendimiento de clasificación.
- Importancia de características:
    - Análisis de variables más relevantes para la predicción.
- Resultados tabulares:
    - DataFrame comparativo con todas las métricas y tiempos.

---

# 2. Importacion de librerias necesarias
--- 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import optuna
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (f1_score, precision_score, recall_score, roc_auc_score, roc_curve)                          
from scipy.stats import randint

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

> **Nota 2:** Los nombres de las columnas se definieron en base a la informacion disponible en `kaggle` sobre este data set: 
[Pima Indians Diabetes Dataset (Kaggle)](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database), ya que en el link de github no se encontraban.

---

**Bloque 1:** Funciones de preprocesamiento de datos.

- **`load_data()`** 
Carga el dataset, define las columnas y lo convierte en un DataFrame con **pandas**. Luego hace una exploración inicial del df determinando las dimenciones y la distribución de clases de lo que sera la variable y.

- **`preprocess_data()`** 
Preprocesa el dataframe reemplazando valores ceros biológicamente incompatibles con la mediana de esa variable, escalando los datos y dividiendo el data set de el sets de entrenamiento y prueba. 

In [ ]:
def load_data(url: str) -> pd.DataFrame:
    """
    Carga un dataset de diabetes desde una URL y realiza una exploración inicial.

    Args:
        url (str): Ruta o URL del archivo CSV que contiene el dataset.

    Returns:
        pd.DataFrame: DataFrame con los datos cargados, incluyendo variables clínicas y el diagnóstico de diabetes.
    """
    column_names = [ # Se definen manualmente debido a que el dataset no las contiene
        "Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
        "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"
    ]
    df = pd.read_csv(url, names=column_names)
    
    print("="*50)
    print("Exploración inicial del dataset")
    print("="*50)
    print("Dimensiones:", df.shape)
    print("\nDistribución de clases:")
    print(df['Outcome'].value_counts(normalize=True)) # Outcome es lo que usaremos como y
    
    # Gráfico de distribución de clases de diabetes
    plt.figure(figsize=(8, 5))
    sns.countplot(x='Outcome', data=df)
    plt.title('Distribución de Diabetes (0: No, 1: Sí)')
    plt.xlabel('Diagnóstico de Diabetes')
    plt.ylabel('Cantidad de Pacientes')
    plt.savefig("distribucion_diabetes.png")
    plt.show()
    
    return df

def preprocess_data(df: pd.DataFrame) -> tuple:
    """
    Preprocesa el dataset de diabetes mediante limpieza, escalado y división para entrenamiento.

    Este proceso incluye:
    - Identificación de valores cero biológicamente inviables en ciertas variables.
    - Reemplazo de ceros por la mediana en dichas variables.
    - Escalado de características numéricas mediante `StandardScaler`.
    - División estratificada del conjunto de datos en entrenamiento y prueba.

    Args:
        df (pd.DataFrame): DataFrame con los datos originales del estudio de diabetes.

    Returns:
        tuple: Una tupla que contiene:
            - X_train (np.ndarray): Datos de entrenamiento escalados.
            - X_test (np.ndarray): Datos de prueba escalados.
            - y_train (pd.Series): Etiquetas de entrenamiento.
            - y_test (pd.Series): Etiquetas de prueba.
            - scaler (StandardScaler): Objeto scaler ajustado para posibles transformaciones futuras.
    """
    # 1. Análisis exploratorio integrado
    print("\n" + "="*50)
    print("Análisis de Valores Cero")
    print("="*50)
    
    # Características que biológicamente NO pueden ser cero
    critical_features = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
    
    # Identificar características con valores cero en critical_features
    features_with_zero = []
    for feature in critical_features:
        zero_count = (df[feature] == 0).sum()
        if zero_count > 0:
            features_with_zero.append(feature)
            print(f"- {feature}: {zero_count} ceros ({zero_count/len(df)*100:.1f}%)")
    
    # 2. Tratamiento de valores cero
    print("\nTratamiento:")
    for feature in features_with_zero:
        original_zero_count = (df[feature] == 0).sum()
        median_value = df[feature].median()
        
        # Reemplazar ceros por la mediana
        df[feature] = df[feature].replace(0, median_value)
        
        # Verificar tratamiento
        new_zero_count = (df[feature] == 0).sum()
        print(f"Reemplazados {original_zero_count} ceros en {feature} con mediana={median_value:.1f}")

    # 3. Estadísticas después del tratamiento
    print("\nResumen estadístico después del tratamiento:")
    print(df[critical_features].describe().loc[['min', 'max']])
    
    # 4. Preparación de datos para modelado
    X = df.drop("Outcome", axis=1)
    y = df["Outcome"]
    
    # 5. Escalado
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 6. División estratificada
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.3, random_state=42, stratify=y
    )
    
    return X_train, X_test, y_train, y_test, scaler

**Bloque 2:** Funciones de modelado.

- **`train_base_model()`** 
Entrena un modelo base con parametros por defecto para futuras comparaciones de metricas, incluyendo el tiempo.

- **`evaluate_model()`** 
Evalúa el rendimiento del modelo dado sobre el conjunto de datos de prueba, incluyendo las métricas a evaluar como F1-Score, precisión, recall y AUC.

In [ ]:
def train_base_model(X_train: np.ndarray, y_train: pd.Series) -> tuple:
    """
    Entrena un modelo base de Random Forest con parámetros por defecto.

    El modelo incluye manejo de desbalanceo de clases mediante `class_weight='balanced'`
    y utiliza todos los núcleos disponibles para acelerar el entrenamiento.

    Args:
        X_train (np.ndarray): Datos de entrenamiento escalados.
        y_train (pd.Series): Etiquetas correspondientes al conjunto de entrenamiento.

    Returns:
        tuple: Una tupla que contiene:
            - model (RandomForestClassifier): Modelo entrenado.
            - train_time (float): Tiempo de entrenamiento en segundos.
    """
    start_time = time.time()
    model = RandomForestClassifier(
        random_state=42,
        class_weight='balanced',  # Manejo de desbalanceo
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    return model, train_time

def evaluate_model(model, X_test: np.ndarray, y_test: pd.Series) -> dict:
    """
    Evalúa el rendimiento del modelo sobre el conjunto de prueba.

    Calcula métricas clave como F1-Score, Precisión, Recall y AUC, 
    utilizando las predicciones del modelo entrenado.

    Args:
        model: Modelo entrenado con un método `predict` y `predict_proba`.
        X_test (np.ndarray): Datos de prueba escalados.
        y_test (pd.Series): Etiquetas verdaderas del conjunto de prueba.

    Returns:
        dict: Diccionario con las métricas de evaluación:
            - "F1-Score": Media armónica entre precisión y recall.
            - "Precisión": Proporción de verdaderos positivos entre los predichos como positivos.
            - "Recall": Proporción de verdaderos positivos entre los casos realmente positivos.
            - "AUC": Área bajo la curva ROC.
    """
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    return {
        "F1-Score": f1_score(y_test, y_pred),
        "Precisión": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_proba)
    }

**Bloque 3:** Funciones de optimización.

- **`optimize_with_gridsearch()`** 
Optimización utilizando **GridSearchCV**.

- **`optimize_with_randomsearch() -> dict`:** 
Optimización utilizando **RandomizedSearchCV**.

- **`optimize_with_optuna() -> dict`:** 
Optimización utilizando **Optuna**.

In [ ]:
def optimize_with_gridsearch(X_train: np.ndarray, y_train: pd.Series) -> tuple:
    """
    Optimiza un modelo Random Forest utilizando búsqueda exhaustiva de hiperparámetros (Grid Search).

    Se evalúan múltiples combinaciones de parámetros mediante validación cruzada,
    maximizando el área bajo la curva ROC (AUC).

    Args:
        X_train (np.ndarray): Datos de entrenamiento escalados.
        y_train (pd.Series): Etiquetas del conjunto de entrenamiento.

    Returns:
        tuple: Una tupla con:
            - best_estimator_ (RandomForestClassifier): Mejor modelo encontrado.
            - best_params_ (dict): Parámetros óptimos seleccionados.
            - time_taken (float): Tiempo de ejecución en segundos.
    """
    param_grid = {
        "n_estimators": [50, 100, 150],
        "max_depth": [5, 10, 15],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ['sqrt', 'log2'],
        "class_weight": ['balanced']
    }
    
    start_time = time.time()
    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=42),
        param_grid,
        scoring='roc_auc',  # Optimizar por AUC
        cv=5,
        n_jobs=-1,
        verbose=0
    )
    grid_search.fit(X_train, y_train)
    time_taken = time.time() - start_time
    
    return grid_search.best_estimator_, grid_search.best_params_, time_taken

def optimize_with_randomsearch(X_train: np.ndarray, y_train: pd.Series) -> tuple:
    """
    Optimiza un modelo Random Forest mediante búsqueda aleatoria de hiperparámetros (Random Search).

    Evalúa una cantidad limitada de combinaciones seleccionadas al azar, 
    optimizando el AUC con validación cruzada.

    Args:
        X_train (np.ndarray): Datos de entrenamiento escalados.
        y_train (pd.Series): Etiquetas del conjunto de entrenamiento.

    Returns:
        tuple: Una tupla con:
            - best_estimator_ (RandomForestClassifier): Mejor modelo encontrado.
            - best_params_ (dict): Parámetros óptimos seleccionados.
            - time_taken (float): Tiempo de ejecución en segundos.
    """
    param_dist = {
        "n_estimators": randint(50, 200),
        "max_depth": randint(5, 30),
        "min_samples_split": randint(2, 20),
        "min_samples_leaf": randint(1, 10),
        "max_features": ['sqrt', 'log2'],
        "class_weight": ['balanced']
    }
    
    start_time = time.time()
    random_search = RandomizedSearchCV(
        RandomForestClassifier(random_state=42),
        param_distributions=param_dist,
        n_iter=30,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        verbose=0,
        random_state=42
    )
    random_search.fit(X_train, y_train)
    time_taken = time.time() - start_time
    
    return random_search.best_estimator_, random_search.best_params_, time_taken

def optimize_with_optuna(X_train: np.ndarray, y_train: pd.Series, n_trials: int = 30) -> tuple:
    """
    Optimiza un modelo Random Forest utilizando búsqueda bayesiana con Optuna.

    Se define una función objetivo que entrena y evalúa modelos en una validación interna
    para evitar sobreajuste, optimizando la métrica AUC.

    Args:
        X_train (np.ndarray): Datos de entrenamiento escalados.
        y_train (pd.Series): Etiquetas del conjunto de entrenamiento.
        n_trials (int, optional): Número de iteraciones (trials) para la optimización. Por defecto 30.

    Returns:
        tuple: Una tupla con:
            - best_model (RandomForestClassifier): Mejor modelo encontrado y reentrenado.
            - best_params (dict): Conjunto de hiperparámetros óptimos.
            - time_taken (float): Tiempo total de ejecución en segundos.
    """
    # División interna para validación (evita data leakage)
    X_train_opt, X_val, y_train_opt, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )
    
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 200),
            "max_depth": trial.suggest_int("max_depth", 5, 30),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ['sqrt', 'log2']),
            "class_weight": 'balanced',
            "random_state": 42,
            "n_jobs": -1
        }
        
        model = RandomForestClassifier(**params)
        model.fit(X_train_opt, y_train_opt)
        y_proba = model.predict_proba(X_val)[:, 1]
        return roc_auc_score(y_val, y_proba)  # Optimizar AUC
    
    start_time = time.time()
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    
    # Entrenar con mejores parámetros usando todos los datos
    best_params = study.best_params
    best_model = RandomForestClassifier(
        **best_params,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
    best_model.fit(X_train, y_train)
    time_taken = time.time() - start_time
    
    return best_model, best_params, time_taken

**Bloque 4:** Funciones de visualización.

- **`plot_combined_metrics()`** 
Crea dos gráficos para comparar los resultados cuantitativos de las métricas de rendimiento y tiempo de ejecución entre el modelo base y los optimizados.

- **`plot_combined_model_analysis() -> dict`:** 
Crea dos gráficos comparar los resultados cualitativos entre los modelos para evaluar ROC e importancia.

In [ ]:
def plot_combined_metrics(results_df: pd.DataFrame):
    """
    Genera una visualización comparativa de métricas de rendimiento y tiempos de ejecución.

    La figura contiene dos subgráficos:
    1. Barras agrupadas con las principales métricas (F1-Score, Precisión, Recall, AUC) por método.
    2. Tiempos de ejecución de cada método en escala logarítmica.

    Args:
        results_df (pd.DataFrame): DataFrame que contiene las métricas evaluadas y el tiempo 
                                   de ejecución por cada método de optimización. Debe incluir 
                                   columnas como 'Método', 'F1-Score', 'Precisión', 'Recall', 
                                   'AUC' y 'Tiempo (s)'.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
    
    # --- Subgráfico 1: Métricas de rendimiento ---
    metrics_df = results_df.drop('Tiempo (s)', axis=1)
    metrics_df.set_index('Método', inplace=True)
    
    # Configuración de colores
    colors = plt.cm.viridis(np.linspace(0, 1, len(metrics_df.columns)))
    
    # Posiciones de las barras
    x = np.arange(len(metrics_df.index))
    width = 0.2
    
    # Dibujar barras para cada métrica
    for i, metric in enumerate(metrics_df.columns):
        ax1.bar(x + i * width, metrics_df[metric], width, label=metric, color=colors[i])
        # Añadir valores numéricos
        for j, value in enumerate(metrics_df[metric]):
            ax1.text(j + i * width, value + 0.02, f'{value:.3f}', 
                    ha='center', fontsize=9)

    # Configurar ejes y leyenda
    ax1.set_title('Comparación de Métricas por Método de Optimización', fontsize=14)
    ax1.set_ylabel('Valor', fontsize=12)
    ax1.set_xticks(x + width * (len(metrics_df.columns) - 1) / 2)
    ax1.set_xticklabels(metrics_df.index, fontsize=10)
    ax1.set_ylim(0, 1)
    ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), 
               fancybox=True, shadow=True, ncol=4, fontsize=10)
    ax1.grid(axis='y', linestyle='--', alpha=0.7)
    
    # --- Subgráfico 2: Tiempos de ejecución ---
    # Usamos escala logarítmica para mejor visualización
    time_df = results_df[['Método', 'Tiempo (s)']].copy()
    time_df['Tiempo (s)'] = time_df['Tiempo (s)'].apply(lambda x: max(x, 0.1))  # Evitar 0 en log
    
    # Gráfico de barras con colores
    colors = plt.cm.plasma(np.linspace(0, 1, len(time_df)))
    bars = ax2.bar(time_df['Método'], time_df['Tiempo (s)'], color=colors)
    ax2.set_yscale('log')
    
    # Añadir valores
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height, f'{height:.1f}s', 
                 ha='center', va='bottom', fontsize=10)
    
    # Configuración
    ax2.set_title('Tiempo de Ejecución por Método', fontsize=14)
    ax2.set_ylabel('Tiempo (segundos, escala log)', fontsize=12)
    ax2.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig("comparacion_cuantitativa")
    plt.show()


def plot_combined_model_analysis(models: dict, X_test: np.ndarray, y_test: pd.Series, 
                                model, feature_names: list):
    """
    Genera una visualización con análisis comparativo de modelos y explicación del modelo final.

    La figura contiene dos subgráficos:
    1. Curvas ROC de distintos modelos para comparar su capacidad discriminativa.
    2. Importancia de características del modelo final entrenado (basado en Random Forest).

    Args:
        models (dict): Diccionario de modelos con nombres como claves y objetos de clasificación como valores.
        X_test (np.ndarray): Datos de prueba escalados.
        y_test (pd.Series): Etiquetas verdaderas del conjunto de prueba.
        model: Modelo final ya entrenado (debe contar con `feature_importances_`).
        feature_names (list): Lista con los nombres de las características originales del dataset.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
    
    # --- Subgráfico 1: Curvas ROC ---
    ax1.plot([0, 1], [0, 1], 'k--', label='Clasificador aleatorio')
    
    # Colores para las curvas
    colors = plt.cm.tab10(np.linspace(0, 1, len(models)))
    
    for i, (name, model_obj) in enumerate(models.items()):
        y_proba = model_obj.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc_score = roc_auc_score(y_test, y_proba)
        ax1.plot(fpr, tpr, label=f'{name} (AUC = {auc_score:.3f})', 
                linewidth=2.5, color=colors[i])
    
    # Configuración
    ax1.set_title('Curvas ROC Comparativas', fontsize=14)
    ax1.set_xlabel('Tasa de Falsos Positivos (FPR)', fontsize=12)
    ax1.set_ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=12)
    ax1.legend(loc='lower right', fontsize=10)
    ax1.grid(True, linestyle='--', alpha=0.7)
    
    # --- Subgráfico 2: Importancia de características ---
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    # Gráfico de barras con colores
    colors = plt.cm.coolwarm(np.linspace(0, 1, len(importances)))
    bars = ax2.bar(range(len(importances)), importances[indices], 
                 color=colors, align='center')
    
    # Configuración de ejes
    ax2.set_title("Importancia de Características", fontsize=14)
    ax2.set_xticks(range(len(importances)))
    ax2.set_xticklabels([feature_names[i] for i in indices], 
                      rotation=45, ha='right', fontsize=10)
    ax2.set_xlabel('Características', fontsize=12)
    ax2.set_ylabel('Importancia Relativa', fontsize=12)
    
    # Añadir valores en las barras
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height, 
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig("comparacion_cualitativa")
    plt.show()

**Bloque 5:** Función de ejecución.
- **`main()`**
Utiliza todas las funciones definidas anteriormente para obtener datos del modelo base y de los modelos optimizados con **GridSearchCV**, **RandomizedSearchCV** y **Optuna**.  

In [ ]:
def main():
    """
    Ejecuta el flujo completo de entrenamiento, optimización y análisis de modelos para detección de diabetes.

    Pasos que realiza:
    1. Carga y exploración inicial del dataset desde una URL.
    2. Preprocesamiento de datos: limpieza, escalado y división en conjuntos.
    3. Entrenamiento de un modelo base con Random Forest.
    4. Optimización de hiperparámetros con tres enfoques: Grid Search, Random Search y Optuna.
    5. Evaluación de rendimiento con métricas como F1-Score, Precisión, Recall y AUC.
    6. Visualización comparativa de métricas y tiempos, además de análisis de importancia de características.

    Returns:
        dict: Diccionario con resultados clave del análisis, incluyendo:
            - 'df' (pd.DataFrame): Dataset original procesado.
            - 'results_df' (pd.DataFrame): Tabla con métricas y tiempos por método.
            - 'models' (dict): Modelos entrenados por nombre.
            - 'feature_names' (list): Lista de nombres de características usadas.
    """
    # 1. Carga de datos
    URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
    df = load_data(URL)
    
    # 2. Preprocesamiento
    X_train, X_test, y_train, y_test, scaler = preprocess_data(df)
    feature_names = df.drop("Outcome", axis=1).columns.tolist()
    
    # 3. Modelo base
    print("\nEntrenando modelo base...")
    base_model, base_time = train_base_model(X_train, y_train)
    base_metrics = evaluate_model(base_model, X_test, y_test)
    print(f"Resultados modelo base:\n{base_metrics}")
    
    # 4. Optimización de hiperparámetros
    print("\nOptimizando con Grid Search...")
    grid_model, grid_params, grid_time = optimize_with_gridsearch(X_train, y_train)
    grid_metrics = evaluate_model(grid_model, X_test, y_test)
    
    print("\nOptimizando con Random Search...")
    random_model, random_params, random_time = optimize_with_randomsearch(X_train, y_train)
    random_metrics = evaluate_model(random_model, X_test, y_test)
    
    print("\nOptimizando con Optuna (Bayesiana)...")
    optuna_model, optuna_params, optuna_time = optimize_with_optuna(X_train, y_train)
    optuna_metrics = evaluate_model(optuna_model, X_test, y_test)
    
    # 5. Comparación de resultados
    results_df = pd.DataFrame({
        "Método": ["Base", "Grid Search", "Random Search", "Optuna"],
        "F1-Score": [base_metrics["F1-Score"], grid_metrics["F1-Score"], 
                     random_metrics["F1-Score"], optuna_metrics["F1-Score"]],
        "Precisión": [base_metrics["Precisión"], grid_metrics["Precisión"], 
                      random_metrics["Precisión"], optuna_metrics["Precisión"]],
        "Recall": [base_metrics["Recall"], grid_metrics["Recall"], 
                  random_metrics["Recall"], optuna_metrics["Recall"]],
        "AUC": [base_metrics["AUC"], grid_metrics["AUC"], 
                random_metrics["AUC"], optuna_metrics["AUC"]],
        "Tiempo (s)": [base_time, grid_time, random_time, optuna_time]
    })
    
    print("\n" + "="*50)
    print("Comparación de Resultados")
    print("="*50)
    print(results_df)
    
    # 6. Visualizaciones
    models_dict = {
        "Base": base_model,
        "Grid Search": grid_model,
        "Random Search": random_model,
        "Optuna": optuna_model
    }
    
    plot_combined_metrics(results_df)
    plot_combined_model_analysis(models_dict, X_test, y_test, optuna_model, feature_names)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
# Ejectura la función main()
if __name__ == "__main__":
    main()

# Análisis de resultados y reflexiones Finales

---

## Hallazgos Clave

1. **Trade-off Precisión vs. Recall**
   - Los modelos optimizados mostraron un descenso leve en precisión (~4%) pero un aumento sustancial en recall (~19%) respecto al modelo base.
   - En contextos clínicos, este balance es positivo: es preferible detectar más casos reales (aunque haya más falsos positivos) que dejar sin diagnosticar pacientes con diabetes.

2. **Eficiencia Computacional**
   - **Grid Search** fue el método más costoso en tiempo (~20 s), siendo 5 a 6 veces más lento que Random Search u Optuna.
   - **Random Search** y **Optuna** lograron mejores resultados con menor tiempo de cómputo, destacando su escalabilidad.
   - **Optuna** ofrece, además, retroalimentación en tiempo real, lo que mejora el proceso iterativo de optimización.

3. **Variables Predictivas Más Relevantes**
   - La glucosa (`Glucose`) fue consistentemente la variable más importante en la predicción.
   - También destacaron el índice de masa corporal (`BMI`) y la edad (`Age`), en línea con evidencia médica sobre factores de riesgo para diabetes tipo II.

---

## Limitaciones y Mejoras Futuras

1. **Manejo del Desbalanceo**
   - Explorar estrategias como pesos de clase, submuestreo/oversampling o SMOTE.

2. **Optimización y Modelos Avanzados**
   - Probar algoritmos adicionales como **XGBoost** o **LightGBM**.
   - Incorporar técnicas de **selección de características** y **validación cruzada anidada** para mayor robustez.

3. **Interpretabilidad Clínica**
   - Analizar interacciones entre variables para mejorar la explicabilidad del modelo frente a profesionales de salud.

---

## Relevancia en Contextos Reales

1. **Aplicación Clínica**
   - Modelos con alto recall ayudan a minimizar falsos negativos, fundamentales en el diagnóstico temprano de diabetes.
   - El costo de un falso positivo (una prueba innecesaria) es menor que el de un diagnóstico perdido.

2. **Procesamiento a Escala**
   - Técnicas como Random Search y Optuna son más adecuadas para conjuntos de datos grandes o en sistemas distribuidos.
   - El uso de técnicas de muestreo puede acelerar iteraciones sin perder rendimiento.

3. **Despliegue y Mantenimiento**
   - Considerar el **monitoreo continuo del desempeño** del modelo en producción.
   - Planificar actualizaciones periódicas con datos recientes.
   - Establecer sistemas de alerta ante cambios significativos en la distribución de datos o el entorno clínico.

---

> **Conclusión Final:**  
> La optimización de hiperparámetros marca una diferencia significativa en el rendimiento de modelos de predicción clínica. En este estudio, Random Search y Optuna mejoraron el **recall en un 19%** respecto al modelo base, manteniendo tiempos computacionales razonables. Estas técnicas son clave para desarrollar sistemas predictivos útiles en el diagnóstico temprano de diabetes tipo II, donde detectar a tiempo puede marcar la diferencia entre tratamiento efectivo y complicaciones severas.